# Deploying a web service to Azure Kubernetes Service (AKS) with automatic Swagger schema generation

In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model

In [ ]:
import azureml.core
from packaging import version

core_version = azureml.core.VERSION
print(core_version)
assert version.parse(core_version) >= version.parse('1.13.0a0')

# Get workspace
Load workspace from an existing subscription

In [ ]:
# using my own subsciption for testing
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Register the model
Register an existing trained model, add descirption and tags.

In [ ]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl", # this points to a local file
                       model_name = "sklearn_regression_model.pkl", # this is the name the model is registered as
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace = ws)

print(model.name, model.description, model.version)

# Create the Environment
Create an environment that the model will be deployed with

In [6]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies 

conda_deps = CondaDependencies.create(conda_packages=['numpy','scikit-learn==0.19.1','scipy'], pip_packages=['azureml-defaults', 'inference-schema==1.1.0'])
myenv = Environment(name='myenv')
myenv.python.conda_dependencies = conda_deps

In [7]:
# using my own ACR for testing, can be removed after release
# myenv.docker._base_image ="ttiye02.azurecr.io/samples/test_image"
# myenv.docker.base_image_registry.address = "ttiye02.azurecr.io"
# myenv.docker.base_image_registry.username = "ttiye02"
# myenv.docker.base_image_registry.password = "+hvBWkb7D8uFSm2t+SIh=F6KGfwffR5y"

# Write the Entry Script
Write the script that will be used to predict on your model and automatically generate Swagger schema. To use schema generation, include the open-source inference-schema package in your dependencies file. For more information on this package, see https://github.com/Azure/InferenceSchema. Define the input and output sample formats in the input_sample and output_sample variables, which represent the request and response formats for the web service. Use these samples in the input and output function decorators on the run() function. The following scikit-learn example uses schema generation.

In [ ]:
%%writefile score.py
import os
import pickle
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.linear_model import Ridge

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType



def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    
# providing sample input for schema generation
numpy_sample_input = NumpyParameterType(np.array([[1,2,3,4,5,6,7,8,9,10],[10,9,8,7,6,5,4,3,2,1]],dtype='float64'))
pandas_sample_input = PandasParameterType(pd.DataFrame({'name': ['Sarah', 'John'], 'age': [25, 26]}))
standard_sample_input = StandardPythonParameterType(0.0)

# This is a nested input sample, any item wrapped by `ParameterType` will be described by schema
sample_input = StandardPythonParameterType({'input1': numpy_sample_input, 
                                            'input2': pandas_sample_input, 
                                            'input3': standard_sample_input})
sample_output = StandardPythonParameterType([1.0, 1.0])

@input_schema('inputs', sample_input)
@output_schema(sample_output)
def run(inputs):
    try:
        data = inputs['input1']
        # data will be convert to target format
        assert isinstance(data, np.ndarray)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

# Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [10]:
from azureml.core.model import InferenceConfig

inf_config = InferenceConfig(entry_script='score.py', environment=myenv)

In [11]:
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'my-aks' 

try:
    
    aks_target = ComputeTarget(workspace = ws, name = aks_name)
    print("found existing cluster")
except ComputeTargetException:
    print("creating new cluster")
    aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)


In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

# Deploy web service to AKS

In [21]:
# Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [ ]:
%%time
aks_service_name ='aks-service1'
import importlib
importlib.reload(azureml.core.model)
aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inf_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

# Test the web service using run method
We test the web sevice by passing data. Run() method retrieves API keys behind the scenes to make sure that call is authenticated.

In [ ]:
%%time
import json

test_sample = json.dumps({'inputs': {'input1': [[1,2,3,4,5,6,7,8,9,10],[10,9,8,7,6,5,4,3,2,1]],
                                    'input2': {'name': ['Sarah', 'John'], 'age': [25, 26]},
                                    'input3': 1.0},
                          'global_parameters': 1.0})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = aks_service.run(input_data = test_sample)
print(prediction)

# Check your swagger

In [24]:
import requests, json
content = requests.get(aks_service.swagger_uri)
swagger = json.loads(content.content)

print(json.dumps(swagger, indent=2))

{
  "swagger": "2.0",
  "info": {
    "title": "aks-service1",
    "description": "API specification for the Azure Machine Learning service aks-service1",
    "version": "1"
  },
  "schemes": [
    "https"
  ],
  "consumes": [
    "application/json"
  ],
  "produces": [
    "application/json"
  ],
  "securityDefinitions": {
    "Bearer": {
      "type": "apiKey",
      "name": "Authorization",
      "in": "header",
      "description": "For example: Bearer abc123"
    }
  },
  "paths": {
    "/api/v1/service/aks-service1/": {
      "get": {
        "operationId": "ServiceHealthCheck",
        "description": "Simple health check endpoint to ensure the service is up at any given point.",
        "responses": {
          "200": {
            "description": "If service is up and running, this response will be returned with the content 'Healthy'",
            "schema": {
              "type": "string"
            },
            "examples": {
              "application/json": "Healthy"
     

# Clean up
Delete the service, image and model.

In [ ]:
aks_service.delete()
model.delete()